In [ ]:
!pip install -q sentence-transformers==4.1.0
!pip install -q einops==0.8.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.7/345.7 kB 11.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import os
import pickle

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# put path of your project folder here
proj_path = '/content/drive/MyDrive/anime_recommendation_system'
os.chdir(proj_path)

In [ ]:
anime = pd.read_pickle('datasets/created_datasets/anime.pkl')
anime_with_synopsis = pd.read_csv('datasets/kaggle_dataset/anime_with_synopsis.csv')
plot_df = pd.read_csv('datasets/created_datasets/plot_df.csv')

In [ ]:
anime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6356 entries, 0 to 6355
Data columns (total 35 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   MAL_ID         6356 non-null   int64 
 1   Name           6356 non-null   object
 2   Score          6356 non-null   object
 3   Genres         6356 non-null   object
 4   English name   6356 non-null   object
 5   Japanese name  6356 non-null   object
 6   Type           6356 non-null   object
 7   Episodes       6356 non-null   object
 8   Aired          6356 non-null   object
 9   Premiered      6356 non-null   object
 10  Producers      6356 non-null   object
 11  Licensors      6356 non-null   object
 12  Studios        6356 non-null   object
 13  Source         6356 non-null   object
 14  Duration       6356 non-null   object
 15  Rating         6356 non-null   object
 16  Ranked         6356 non-null   object
 17  Popularity     6356 non-null   int64 
 18  Members        6356 non-null

In [ ]:
anime = anime.copy()

In [ ]:
anime['synopsis'] = anime['MAL_ID'].map(anime_with_synopsis.set_index('MAL_ID')['sypnopsis'])
anime['synopsis']

,synopsis
0,"In the year 2071, humanity has colonized sever..."
1,"other day, another bounty—such is the life of ..."
2,"Vash the Stampede is the man with a $$60,000,0..."
3,ches are individuals with special powers like ...
4,It is the dark century and the people are suff...
...,...
6351,"The first episode of ""season"" one was pre-aire..."
6352,cap of all 12 episodes of Tonikaku Kawaii .
6353,summary of season one of Tensei shitara Slime ...
6354,Valentine's Day special for Kimetsu no Yaiba ....


In [ ]:
anime['synopsis_edited'] = anime['synopsis'].str.replace('\n', ' ')

In [ ]:
anime['synopsis_edited'].value_counts().head(10)

,count
synopsis_edited,
No synopsis information has been added to this title. Help improve our database by adding a synopsis here .,26
The story follows a boy named Quon and others who suddenly wake up with supernatural powers.,5
"Set in the land of Yuguto, the people thought that the land they live is the size of the world, but it’s actually divided into several areas, each with a respective king. Even though there were small wars among areas, the kings had roundtable meetings to maintain peace and balance. It was until a dark group of dark monsters appeared... (Source: MAL News)",3
Special episodes added to DVDs and Blu-rays.,3
"For high schooler Kei—and for at least forty-six others—immortality comes as the nastiest surprise ever. Sadly for Kei, such a feat doesn't make him a superhero. In the eyes of both the general public and governments, he's a rare specimen who needs to be hunted down and handed over to scientists to be experimented on for life—a demi-human who must die a thousand deaths for the benefit of humanity. (Source: Vertical)",3
BD/DVD specials.,3
"atsutaka Nanako is a normal university student, who happened to enroll in an Abnormal Physiology seminar. She tries to keep her mental state sound, but other abnormal classmates start to influence her personality... (Source: MU)",2
Picture dramas bundled with the Blu-ray and DVD volumes.,2
Unaired specials included in the Blu-ray and DVD release.,2


In [ ]:
remove_synopsis = anime['synopsis_edited'].value_counts().head(1).index.tolist()
remove_synopsis

['No synopsis information has been added to this title. Help improve our database by adding a synopsis here .']

In [ ]:
anime.loc[anime['synopsis_edited'].isin(remove_synopsis), 'synopsis_edited'] = np.nan

In [ ]:
anime['synopsis_edited'].value_counts().head(5)

,count
synopsis_edited,
The story follows a boy named Quon and others who suddenly wake up with supernatural powers.,5
Special episodes added to DVDs and Blu-rays.,3
BD/DVD specials.,3
"For high schooler Kei—and for at least forty-six others—immortality comes as the nastiest surprise ever. Sadly for Kei, such a feat doesn't make him a superhero. In the eyes of both the general public and governments, he's a rare specimen who needs to be hunted down and handed over to scientists to be experimented on for life—a demi-human who must die a thousand deaths for the benefit of humanity. (Source: Vertical)",3
"Set in the land of Yuguto, the people thought that the land they live is the size of the world, but it’s actually divided into several areas, each with a respective king. Even though there were small wars among areas, the kings had roundtable meetings to maintain peace and balance. It was until a dark group of dark monsters appeared... (Source: MAL News)",3


In [ ]:
anime['synopsis_edited'].isna().sum()

np.int64(26)

In [ ]:
anime['synopsis_edited'][anime['synopsis_edited'].notna()].str.split(' ').apply(lambda x: len(x)).value_counts().sort_index().head(20)

,count
synopsis_edited,
2,18
3,8
4,26
5,47
6,81
7,70
8,84
9,75
10,59


In [ ]:
synp_not_na = anime['synopsis_edited'][anime['synopsis_edited'].notna()]
synp_not_na

,synopsis_edited
0,"In the year 2071, humanity has colonized sever..."
1,"other day, another bounty—such is the life of ..."
2,"Vash the Stampede is the man with a $$60,000,0..."
3,ches are individuals with special powers like ...
4,It is the dark century and the people are suff...
...,...
6351,"The first episode of ""season"" one was pre-aire..."
6352,cap of all 12 episodes of Tonikaku Kawaii .
6353,summary of season one of Tensei shitara Slime ...
6354,Valentine's Day special for Kimetsu no Yaiba ....


In [ ]:
short_sysp = synp_not_na[synp_not_na.str.split(' ').apply(lambda x: len(x) <= 30)].index.tolist()
len(short_sysp)

1225

In [ ]:
anime.loc[short_sysp, 'synopsis_edited'] = np.nan

In [ ]:
anime['synopsis_edited'].isna().sum()

np.int64(1251)

In [ ]:
anime['plot'] = anime['MAL_ID'].map(plot_df.set_index('MAL_ID')['Plot'])
anime['plot']

,plot
0,"In the year 2071, roughly fifty years after an..."
1,"A few days before Halloween on Mars, Faye witn..."
2,"In a distant future, a man known as ""Vash the ..."
3,"Robin Sena is a ""craft user"", born in Japan an..."
4,"Set in the Dark Ages, a term referring to the ..."
...,...
6351,NaN
6352,NaN
6353,NaN
6354,NaN


In [ ]:
anime['plot_edited'] = anime['plot']

In [ ]:
anime['plot_edited'][anime['plot_edited'].notna()].str.split(' ').apply(lambda x: len(x)).value_counts().sort_index().head(20)

,count
plot_edited,
5,1
10,1
13,1
14,1
15,1
16,1
18,2
19,1
20,1


In [ ]:
plot_not_na = anime['plot_edited'][anime['plot_edited'].notna()]
plot_not_na

,plot_edited
0,"In the year 2071, roughly fifty years after an..."
1,"A few days before Halloween on Mars, Faye witn..."
2,"In a distant future, a man known as ""Vash the ..."
3,"Robin Sena is a ""craft user"", born in Japan an..."
4,"Set in the Dark Ages, a term referring to the ..."
...,...
6339,The story focuses on the growing relationship ...
6340,"In Okinawa, a group of hardcore skaters partic..."
6342,"The series follows Ai Ohto, a junior high scho..."
6346,High school girl Yuri Honjō is transported to ...


In [ ]:
short_plot = plot_not_na[plot_not_na.str.split(' ').apply(lambda x: len(x) <= 30)].index.tolist()
len(short_plot)

27

In [ ]:
anime.loc[short_plot, 'plot_edited'] = np.nan

In [ ]:
anime['plot'][anime['plot'].notna()].str.split(' ').apply(lambda x: len(x) > 500).sum()

np.int64(540)

In [ ]:
def comparison_func(df):

    if pd.isna(df['plot_edited']):
        return df['synopsis_edited']

    if pd.isna(df['synopsis_edited']):
        return df['plot_edited']

    len1 = len(df['plot_edited'].split(' '))
    len2 = len(df['synopsis_edited'].split(' '))
    if len1 >= len2:
        return df['plot_edited']
    else:
        return df['synopsis_edited']


In [ ]:
comp_results = anime.apply(comparison_func, axis = 1)
comp_results

,0
0,"In the year 2071, roughly fifty years after an..."
1,"A few days before Halloween on Mars, Faye witn..."
2,"In a distant future, a man known as ""Vash the ..."
3,"Robin Sena is a ""craft user"", born in Japan an..."
4,"Set in the Dark Ages, a term referring to the ..."
...,...
6351,NaN
6352,NaN
6353,NaN
6354,Valentine's Day special for Kimetsu no Yaiba ....


In [ ]:
anime['final_plot'] = comp_results

In [ ]:
anime['final_plot'].notna().sum()

np.int64(5144)

In [ ]:
content_df = anime.loc[anime['final_plot'].notna(),['MAL_ID', 'final_plot', 'Name']]

In [ ]:
content_df['final_plot'].shape

(5144,)

In [ ]:
content_df.reset_index(drop = True, inplace = True)

In [ ]:
content_df.head()

,MAL_ID,final_plot,Name
0,1,"In the year 2071, roughly fifty years after an...",Cowboy Bebop
1,5,"A few days before Halloween on Mars, Faye witn...",Cowboy Bebop: Tengoku no Tobira
2,6,"In a distant future, a man known as ""Vash the ...",Trigun
3,7,"Robin Sena is a ""craft user"", born in Japan an...",Witch Hunter Robin
4,8,"Set in the Dark Ages, a term referring to the ...",Bouken Ou Beet


In [ ]:
model = SentenceTransformer('Alibaba-NLP/gte-large-en-v1.5', trust_remote_code=True)

In [ ]:
content_df['plot_embeddings'] = content_df['final_plot'].apply(lambda x: model.encode(x, convert_to_tensor=False))

In [ ]:
import pickle
save_path = 'datasets/created_datasets/content_df_model.pkl'
with open(save_path, 'wb') as f:
    pickle.dump(content_df, f)